In [3]:
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from utils import *
from model import *
from process import *
import pickle
import itertools
import pickle
import os

In [4]:
data='cora'
adj, features, labels,idx_train,idx_val,idx_test = load_citation(data)
#splitstr = 'splits/'+data+'_split_0.6_0.2_'+str(0)+'.npz'
#adj, features, labels, idx_train, idx_val, idx_test, num_features, num_labels = full_load_data(data,splitstr)
print(adj.shape)
print(features.shape)
print(labels.shape)
print(idx_train.shape)
print(idx_val.shape)
print(idx_test.shape)

torch.Size([2708, 2708])
torch.Size([2708, 1433])
torch.Size([2708])
torch.Size([140])
torch.Size([500])
torch.Size([1000])


C:\Users\ayday\Desktop\Master_Thesis\experiment\utils.py:87: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


In [5]:
cudaid = "cuda"
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device).coalesce()

In [6]:
n=features.shape[0]
feature_size=features.shape[1]
num_classes = len(torch.unique(labels))




In [23]:
'''
[AH,A,X]
results
combinations
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.4, 'nhid_list': [128, 512, 128], 'accuracy': 0.792}
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [16, 32], 'accuracy': 0.778}
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.5, 'nhid_list': [128, 256, 128], 'accuracy': 0.786}

'''
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features+n+nfeat, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        support = torch.cat((torch.mm(adj.to_dense(), input),adj.to_dense(),x),1)
        output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)


In [ ]:
[H-AH, A, X]
{'lr': 0.05, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [], 'accuracy': 0.64}
results_1
intersting all the good acc without hidden layer
'''
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features+n+nfeat, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        #support = torch.cat((torch.mm(adj.to_dense(), input),adj.to_dense(),x),1)
        #output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

        support = torch.cat((input - torch.mm(adj.to_dense(), input),adj.to_dense(),x),1)
        output = torch.mm(support, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)
'''

In [20]:
'''
AH+H
skip connection
results_2
{'lr': 0.001, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [512, 1024, 512], 'accuracy': 0.8200000000000001}
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.5, 'nhid_list': [256, 512], 'accuracy': 0.81}
'''
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
       
        
        support = 0.9*torch.mm(adj.to_dense(), input) + 0.1*input
        output = torch.mm(support, self.weight)


       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)


In [ ]:
[X,AH+H]
{'lr': 0.001, 'weight_decay': 0.001, 'dropout': 0.4, 'nhid_list': [128, 512, 128], 'accuracy': 0.812}
{'lr': 0.01, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [64, 128, 64], 'accuracy': 0.81}
results3
'''
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(nfeat+in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        support = torch.cat((x,torch.mm(adj.to_dense(), input) + input),1)
        output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)
'''

In [15]:
#[AH,X]
results4
#{'lr': 0.005, 'weight_decay': 0.005, 'dropout': 0.3, 'nhid_list': [512, 1024, 512], 'accuracy': 0.8180000000000001}
{'lr': 0.001, 'weight_decay': 0.0005, 'dropout': 0.5, 'nhid_list': [512, 1024, 512], 'mean_accuracy': 0.8113333333333334, 'std_accuracy': 0.011813363431112911}
'''
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(nfeat+in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        support = torch.cat((torch.mm(adj.to_dense(), input),x),1)
        output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)
'''

In [24]:

import time
import itertools
import os
import pickle
import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F

# Define your train, validate, and test functions
def train(model, optimizer, features, adj, labels, idx_train):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(), acc_train.item()

def validate(model, features, adj, labels, idx_val):
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(), acc_val.item()

def test(model, features, adj, labels, idx_test, checkpt_file):
    model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(), acc_test.item()

def load_previous_results(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    return []

def save_results(file_path, results):
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)

# Hyperparameters
nfeat = features.shape[1]
nclass = num_classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hyperparams = {
    'lr': [0.05, 0.01, 0.005, 0.001],
    'weight_decay': [5e-4, 5e-3, 1e-4, 1e-3],
    'dropout': [0.5, 0.3, 0.4],
    'nhid_list': [[], [64],[64,64],[256, 512], [16, 32, 16], [64, 128, 64], [512, 1024, 512], [128, 512, 128]]
}

# File paths for saving results and combinations
results_file_path = 'auto_results/results.pkl'
combinations_file_path = 'auto_results/combinations.pkl'

# To store results
results = []

# To store already run combinations
run_combinations = set()

# Load previously run combinations if available
results = load_previous_results(results_file_path)
for result in results:
     run_combinations.add((result['lr'], result['weight_decay'], result['dropout'], tuple(result['nhid_list'])))

for lr, weight_decay, dropout, nhid_list in itertools.product(
    hyperparams['lr'], hyperparams['weight_decay'], hyperparams['dropout'], hyperparams['nhid_list']
):
    combination = (lr, weight_decay, dropout, tuple(nhid_list))
    
    if combination in run_combinations:
        print(f"Skipping already run combination: {combination}")
        continue
    
    run_combinations.add(combination)
    
    accuracies = []
    for _ in range(3):  # Run each combination 3 times
        model = my_GCN(nfeat, nhid_list, nclass, dropout)
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        
        epochs = 100
        patience = 10
        best = 999999999
        best_epoch = 0
        acc = 0
        t_total = time.time()
        bad_counter = 0

        for epoch in range(epochs):
            loss_tra, acc_tra = train(model, optimizer, features, adj, labels, idx_train)
            loss_val, acc_val = validate(model, features, adj, labels, idx_val)
            if (epoch+1) % 1 == 0: 
                pass  # You can print progress here if needed

            if loss_val < best:
                best = loss_val
                best_epoch = epoch
                acc = acc_val
                bad_counter = 0
            else:
                bad_counter += 1

            if bad_counter == patience:
                break

        accuracies.append(acc)
    
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    print(mean_acc)
    
    results.append({
        'lr': lr,
        'weight_decay': weight_decay,
        'dropout': dropout,
        'nhid_list': nhid_list,
        'mean_accuracy': mean_acc,
        'std_accuracy': std_acc
    })

# Sort results by mean accuracy in descending order
results = sorted(results, key=lambda x: x['mean_accuracy'], reverse=True)

print("Hyperparameters and accuracies sorted by mean accuracy:")
for result in results:
    print(result)

# Save results and run_combinations for future use
save_results(results_file_path, results)
save_results(combinations_file_path, list(run_combinations))


0.7313333333333333
0.7566666666666667
0.7680000000000001
0.7639999999999999
0.7566666666666667
0.7406666666666667
0.7493333333333334
0.7493333333333334
0.7346666666666666
0.7520000000000001
0.754
0.7646666666666667
0.7319999999999999
0.7479999999999999
0.7346666666666666
0.7399999999999999
0.7333333333333334
0.7593333333333333
0.746
0.7486666666666667
0.7606666666666667
0.7426666666666666
0.7353333333333333
0.754
0.6406666666666667
0.7706666666666667
0.7399999999999999
0.7380000000000001
0.7160000000000001
0.7559999999999999
0.7493333333333334
0.7513333333333333
0.65
0.7726666666666667
0.7360000000000001
0.774
0.7046666666666667
0.7566666666666667
0.738
0.766
0.6193333333333334
0.7786666666666667
0.7566666666666667
0.7653333333333334
0.7473333333333333
0.7546666666666667
0.7593333333333333
0.7466666666666667
0.7080000000000001
0.7753333333333333
0.7760000000000001
0.7526666666666667
0.7453333333333333
0.7326666666666667
0.6986666666666667
0.7160000000000001
0.7186666666666666
0.7606666